In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame

import matplotlib.pyplot as plt
import seaborn as sns

# Part I: Read in data for zip codes 
* List of all US zip codes can be found at https://www.unitedstateszipcodes.org/zip-code-database/

In [3]:
path=r'C:\Users\aleci\Documents\Project 4 - Study of Home Price Changes in Soutern California\data\\'

zipcode=pd.read_excel(path+'zip_code_database.xls')

In [4]:
zipcode.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631.0,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631.0,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,787939.0,NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,787939.0,NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787.0,NaN,US,18.43,-67.15,0


#### We will analyze the mortgage data in southern California only

In [5]:
# Select zip code for California only
zipcode_CA=zipcode[zipcode['state']=='CA']
zipcode_CA.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
38211,90001,STANDARD,0,Los Angeles,"Firestone Park, Firestone Pk",NaN,CA,Los Angeles County,America/Los_Angeles,3.230000e+02,NaN,US,33.97,-118.24,49200
38212,90002,STANDARD,0,Los Angeles,Watts,NaN,CA,Los Angeles County,America/Los_Angeles,3.235620e+05,NaN,US,33.94,-118.24,45530
38213,90003,STANDARD,0,Los Angeles,NaN,Broadway Manchester,CA,Los Angeles County,America/Los_Angeles,3.230000e+02,NaN,US,33.96,-118.27,62390
38214,90004,STANDARD,0,Los Angeles,Oakwood,NaN,CA,Los Angeles County,America/Los_Angeles,3.230000e+02,NaN,US,34.07,-118.31,46650
38215,90005,STANDARD,0,Los Angeles,Sanford,NaN,CA,Los Angeles County,America/Los_Angeles,2.133103e+17,NaN,US,34.06,-118.31,27590


In [6]:
#  list of all counties in California
zipcode_CA.groupby('county')['zip'].count()

county
Alameda County             70
Alpine County               3
Amador County              12
Butte County               24
Calaveras County           21
Colusa County               7
Contra Costa County        52
Del Norte County            6
El Dorado County           30
Fresno County             101
Glenn County                7
Humboldt County            38
Imperial County            17
Inyo County                13
Kern County                69
Kings County                9
Lake County                15
Lassen County              15
Los Angeles County        528
Madera County              13
Marin County               42
Mariposa County             8
Mendocino County           28
Merced County              22
Modoc County               10
Mono County                 7
Monterey County            35
Napa County                12
Nevada County              12
Orange County             149
Placer County              32
Plumas County              16
Riverside County          101
Sac

* Find names for all counties in southern California at https://en.wikipedia.org/wiki/Category:Counties_in_Southern_California
* In Fannie Mae data, only the first 3 digits are retained as ZIP.  For Example, 91016 and 91017 are expressed as 910.
* To merge with Fannie Mae files later, we need to trim the zip code by deleting the last two digits. 

In [7]:
# select the zip codes in southern California
socal_county=["Los Angeles County", "Orange County", "Reverside County", "San Bernadino County", "Ventura County",
             "San Diego County", "Imperial County", "Kern County", "Santa Barbara County", "San Luis Obispo County"]
zipcode_socal=zipcode_CA[zipcode_CA['county'].isin(socal_county)]

# deleting the last two digits in the 5-digit zip code
zipcode_socal['ZIP']=(zipcode['zip']/100).apply(np.floor)  # e.g., zip code 91016 becomnes 910.16 and is floored as 910
print(zipcode_socal.shape)
zipcode_socal.drop_duplicates(subset='ZIP', keep='first', inplace=True)
print(zipcode_socal.shape)

(1076, 16)
(31, 16)


C:\Users\aleci\AppData\Local\Temp/ipykernel_10320/3335020897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipcode_socal['ZIP']=(zipcode['zip']/100).apply(np.floor)  # e.g., zip code 91016 becomnes 910.16 and is floored as 910
C:\Users\aleci\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
zipcode_socal.groupby('county')['zip'].count()

county
Imperial County            1
Kern County                3
Los Angeles County        18
Orange County              3
San Diego County           3
San Luis Obispo County     1
Santa Barbara County       1
Ventura County             1
Name: zip, dtype: int64

In [9]:
zipcode_socal.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population,ZIP
38211,90001,STANDARD,0,Los Angeles,"Firestone Park, Firestone Pk",NaN,CA,Los Angeles County,America/Los_Angeles,323.0,NaN,US,33.97,-118.24,49200,900.0
38306,90101,STANDARD,1,Los Angeles,NaN,"Bulk Mail Center, La",CA,Los Angeles County,America/Los_Angeles,213.0,NaN,US,33.99,-118.16,0,901.0
38311,90201,STANDARD,0,Bell Gardens,"Bell, Cudahy",NaN,CA,Los Angeles County,America/Los_Angeles,323.0,NaN,US,33.97,-118.15,86140,902.0
38361,90301,STANDARD,0,Inglewood,NaN,NaN,CA,Los Angeles County,America/Los_Angeles,323310424.0,NaN,US,33.95,-118.36,30730,903.0
38376,90401,STANDARD,0,Santa Monica,NaN,NaN,CA,Los Angeles County,America/Los_Angeles,310424.0,NaN,US,34.01,-118.49,6160,904.0


# Part II: Read in Fannie Mae morgage files from 2020 and 2021
### The following code will take 6-7 hours to run. 
* The files will be merged with socal zip codes to get morgtage data for southern California

In [10]:
columns=["POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "SELLER", "SERVICER",
                        "MASTER_SERVICER", "ORIG_RATE", "CURR_RATE", "ORIG_UPB", "ISSUANCE_UPB",
                        "CURRENT_UPB", "ORIG_TERM", "ORIG_DATE", "FIRST_PAY", "LOAN_AGE",
                        "REM_MONTHS", "ADJ_REM_MONTHS", "MATR_DT", "OLTV", "OCLTV",
                        "NUM_BO", "DTI", "CSCORE_B", "CSCORE_C", "FIRST_FLAG", "PURPOSE",
                        "PROP", "NO_UNITS", "OCC_STAT", "STATE", "MSA", "ZIP", "MI_PCT",
                        "PRODUCT", "PPMT_FLG", "IO", "FIRST_PAY_IO", "MNTHS_TO_AMTZ_IO",
                        "DLQ_STATUS", "PMT_HISTORY", "MOD_FLAG", "MI_CANCEL_FLAG", "Zero_Bal_Code",
                        "ZB_DTE", "LAST_UPB", "RPRCH_DTE", "CURR_SCHD_PRNCPL", "TOT_SCHD_PRNCPL",
                        "UNSCHD_PRNCPL_CURR", "LAST_PAID_INSTALLMENT_DATE", "FORECLOSURE_DATE",
                        "DISPOSITION_DATE", "FORECLOSURE_COSTS", "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
                        "ASSET_RECOVERY_COSTS", "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
                        "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", "NET_SALES_PROCEEDS",
                        "CREDIT_ENHANCEMENT_PROCEEDS", "REPURCHASES_MAKE_WHOLE_PROCEEDS",
                        "OTHER_FORECLOSURE_PROCEEDS", "NON_INTEREST_BEARING_UPB", "PRINCIPAL_FORGIVENESS_AMOUNT",
                        "ORIGINAL_LIST_START_DATE", "ORIGINAL_LIST_PRICE", "CURRENT_LIST_START_DATE",
                        "CURRENT_LIST_PRICE", "ISSUE_SCOREB", "ISSUE_SCOREC", "CURR_SCOREB",
                        "CURR_SCOREC", "MI_TYPE", "SERV_IND", "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
                        "CUMULATIVE_MODIFICATION_LOSS_AMOUNT", "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
                        "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS", "HOMEREADY_PROGRAM_INDICATOR",
                        "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT", "RELOCATION_MORTGAGE_INDICATOR",
                        "ZERO_BALANCE_CODE_CHANGE_DATE", "LOAN_HOLDBACK_INDICATOR", "LOAN_HOLDBACK_EFFECTIVE_DATE",
                        "DELINQUENT_ACCRUED_INTEREST", "PROPERTY_INSPECTION_WAIVER_INDICATOR",
                        "HIGH_BALANCE_LOAN_INDICATOR", "ARM_5_YR_INDICATOR", "ARM_PRODUCT_TYPE",
                        "MONTHS_UNTIL_FIRST_PAYMENT_RESET", "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
                        "INTEREST_RATE_CHANGE_DATE", "PAYMENT_CHANGE_DATE", "ARM_INDEX",
                        "ARM_CAP_STRUCTURE", "INITIAL_INTEREST_RATE_CAP", "PERIODIC_INTEREST_RATE_CAP",
                        "LIFETIME_INTEREST_RATE_CAP", "MARGIN", "BALLOON_INDICATOR",
                        "PLAN_NUMBER", "FORBEARANCE_INDICATOR", "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
                        "DEAL_NAME", "RE_PROCS_FLAG", "ADR_TYPE", "ADR_COUNT", "ADR_UPB"]

In [14]:
# columns/variables to keep 
columns_to_keep=["ACT_PERIOD", "POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "ORIG_RATE", "CURR_RATE", "ORIG_UPB", "ISSUANCE_UPB",
                 "CURRENT_UPB", "ORIG_TERM", "ORIG_DATE", "OLTV", "OCLTV",
                 "DTI", "CSCORE_B", "CSCORE_C", "PURPOSE",
                 "PROP", "NO_UNITS", "STATE", "MSA", "ZIP", "MI_PCT", "DLQ_STATUS", "ORIGINAL_LIST_PRICE",
                 "CURRENT_LIST_PRICE"]

# You can adjust the chunk size, depending on the capacity of your computer
chunk_size=10000

state_name='CA'

destine=r'C:\Users\aleci\Documents\Project 4 - Study of Home Price Changes in Soutern California\data\monthly data\\'

def getfile(year_quarter, year_month, period):
    mylist=[]
    for chunk in pd.read_csv(path+'20'+str(year_quarter)+'.csv', sep="|", names=columns, header=None, 
                             low_memory=False, error_bad_lines=False, chunksize=chunk_size):
        chunk=chunk[chunk['ACT_PERIOD']==period]
        mylist.append(chunk)
    file=pd.concat(mylist, axis=0)  
    # retain one record per loan 
    file=file.drop_duplicates(subset=['POOL_ID', 'LOAN_ID'], keep='first')
    # keep useful columns only to further limit file size
    file=file[columns_to_keep]
    # select California data only to limit the file size
    file=file[file['STATE']==state_name]
    
     # Merge with the list of zip codes (first 3 digits) in southern California
    file=pd.merge(file, zipcode_socal, on='ZIP', how='inner')
    # Export the output file to csv so that we do not need to repeat the same exercise when we need to use the same data next time
    # make a subfolder called year_month to store the output sample
    file.to_csv(destine+'20'+str(str(year_month))+'_socal.csv', index=False)
    return file

CA_2020q1=getfile("20Q1", 2001, 12020)

C:\Users\aleci\AppData\Local\Temp/ipykernel_10320/3097328887.py:36: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2020q1=getfile("20Q1", 2001, 12020)


In [38]:
CA_2020q1=getfile("20Q1", 2002, 22020)
CA_2020q1=getfile("20Q1", 2003, 32020)

CA_2020q2=getfile("20Q2", 2004, 42020)
CA_2020q2=getfile("20Q2", 2005, 52020)
CA_2020q2=getfile("20Q2", 2006, 62020)

CA_2020q3=getfile("20Q3", 2007, 72020)
CA_2020q3=getfile("20Q3", 2008, 82020)
CA_2020q3=getfile("20Q3", 2009, 92020)

CA_2020q4=getfile("20Q4", 2010, 102020)
CA_2020q4=getfile("20Q4", 2011, 112020)
CA_2020q4=getfile("20Q4", 2012, 122020)

CA_2021q1=getfile("21Q1", 2101, 12021)
CA_2021q1=getfile("21Q1", 2102, 22021)
CA_2021q1=getfile("21Q1", 2103, 32021)

CA_2021q2=getfile("21Q2", 2104, 42021)
CA_2021q2=getfile("21Q2", 2105, 52021)
CA_2021q2=getfile("21Q2", 2106, 62021)

CA_2021q3=getfile("21Q3", 2107, 72021)
CA_2021q3=getfile("21Q3", 2108, 82021)
CA_2021q3=getfile("21Q3", 2109, 92021)

CA_2021q4=getfile("21Q4", 2110, 102021)
CA_2021q4=getfile("21Q4", 2111, 112021)
CA_2021q4=getfile("21Q4", 2112, 122021)

C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/961356683.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2020q1=getfile("20Q1", 2001, 12020)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/961356683.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2020q1=getfile("20Q1", 2002, 22020)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/961356683.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2020q1=getfile("20Q1", 2003, 32020)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/961356683.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2020q3=getfile("20Q3", 2007, 72020)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/961356683.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be remove

In [39]:
CA_2018q1=getfile("18Q1", 1801, 12018)
CA_2018q1=getfile("18Q1", 1802, 22018)
CA_2018q1=getfile("18Q1", 1803, 32018)

CA_2018q2=getfile("18Q2", 1804, 42018)
CA_2018q2=getfile("18Q2", 1805, 52018)
CA_2018q2=getfile("18Q2", 1806, 62018)

CA_2018q3=getfile("18Q3", 1807, 72018)
CA_2018q3=getfile("18Q3", 1808, 82018)
CA_2018q3=getfile("18Q3", 1809, 92018)

CA_2018q4=getfile("18Q4", 1810, 102018)
CA_2018q4=getfile("18Q4", 1811, 112018)
CA_2018q4=getfile("18Q4", 1812, 122018)

CA_2019q1=getfile("19Q1", 1901, 12019)
CA_2019q1=getfile("19Q1", 1902, 22019)
CA_2019q1=getfile("19Q1", 1903, 32019)

CA_2019q2=getfile("19Q2", 1904, 42019)
CA_2019q2=getfile("19Q2", 1905, 52019)
CA_2019q2=getfile("19Q2", 1906, 62019)

CA_2019q3=getfile("19Q3", 1907, 72019)
CA_2019q3=getfile("19Q3", 1908, 82019)
CA_2019q3=getfile("19Q3", 1909, 92019)

CA_2019q4=getfile("19Q4", 1910, 102019)
CA_2019q4=getfile("19Q4", 1911, 112019)
CA_2019q4=getfile("19Q4", 1912, 122019)

CA_2022q1=getfile("22Q1", 2201, 12022)
CA_2022q1=getfile("22Q1", 2202, 22022)
CA_2022q1=getfile("22Q1", 2203, 32022)

C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/395823553.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2018q1=getfile("18Q1", 1801, 12018)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/395823553.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2018q1=getfile("18Q1", 1802, 22018)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/395823553.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2018q1=getfile("18Q1", 1803, 32018)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/395823553.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  CA_2018q2=getfile("18Q2", 1804, 42018)
C:\Users\aleci\AppData\Local\Temp/ipykernel_3984/395823553.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed